<h1>Logistische Regression mit Tensorflow anhand der MNIST Ziffern</h1>

<p>Bisher haben wir mit numpy die ML Modelle und deren Ableitungen manuell programmmiert. Gute Machine Learning Frameworks können uns einiges an Arbeit abnehmen. Dank automatischer Differenzierung müssen wir uns zum Beispiel keine Gedanken mehr machen, wie die Ableitung einer Funktion aussieht. Es reicht aus, wenn wir den sogenannten Vorwärtspass bis hin zur Fehlerfunktion hinschreiben und den Rest dem Framework überlassen. In den nächsten Übungen wollen wir mit der Bibliothek Tensorflow arbeiten, welche sowohl auf der CPU als auch auf der GPU läuft. Es wird empfohlen, das Framework mit Grafikkartenunterstützung zu installieren. Voraussetzung ist eine NVIDIA-Grafikkarte, die im Labor vorzufinden ist.</p>

<p>Dieses Jupyter Notebook steht wieder zum <a href="05_Tensorflow_Regression_MNIST_Vorlage.ipynb" target="_blank">Download</a> bereit.</p>

<hr />
<h2>Vorbereitung</h2>

<p>Die bisher verwendete numpy Bibliothek arbeitet <strong>imperative </strong>und f&uuml;hrt jede geschrieben Zeile sofort aus. Niedergeschriebene Formeln werden sofort berechnet und die Ergebnisse stehen im Anschluss zur Verf&uuml;gung. <strong>Deklarative </strong>Programmierung hingegen, beschreibt nur was berechnet werden soll, aber nicht wie und wann. Python selbst ist zwar imperativ, aber der Computation Graph von Tensorflow arbeitet deklarativ. Wir unterscheiden deshalb zwischen zwei Phasen bei der Programmierung mit Tensorflow. Zun&auml;chst erstellen wir den Graphen, was f&uuml;r uns aussieht wie eine imperative Schreibweise ist in Wirklichkeit eine deklarative. Im zweiten Schritt erstellen wir eine Session, die den Graphen ausf&uuml;hrt und uns dessen Ergebnis ausgibt.</p>

In [ ]:
!pip install tensorflow==1.15.0

In [ ]:
!pip install tensorflow-gpu==1.15.0

In [ ]:
!pip install deep-teaching-commons

<hr />

<h2>MNIST mit Logistischer Regression in Tensorflow</h2>

<p>Implementieren Sie die logistische Regression in Tensorflow, die die Ziffern der MNIST Bilder vorhersagen kann. Die Aufgabe ist identisch zu &Uuml;bung 2. Zunächst die Daten normalisieren und in Trainungs- und Testdaten einteilen.</p>

In [ ]:
from shutil import copyfileobj
from sklearn.datasets.base import get_data_home
from deep_teaching_commons.data.fundamentals import mnist
from sklearn.utils import check_random_state
from sklearn.preprocessing import OneHotEncoder

import os
import urllib

import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
X_train, y_train, X_test, y_test = mnist.Mnist().get_all_data(normalized=True)

# only shuffle train dataset
random_state = check_random_state(0)
permutation = random_state.permutation(X_train.shape[0])
X_train = X_train[permutation]
y_train = y_train[permutation]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

Die y-Daten One-Hot kodieren

In [ ]:
enc = OneHotEncoder()

y_train_hot = enc.fit_transform(np.expand_dims(y_train, axis=1)).toarray()
y_test_hot = enc.fit_transform(np.expand_dims(y_test, axis=1)).toarray()
print(y_train_hot.shape, y_test_hot.shape)

<hr />
<h2>Training</h2>

<p>Überführen Sie die logistischen Regression von Übung 3 in eine Tensorflow-Version. Verwenden Sie Softmax als Aktivierungsfunktion und die Cross-Entropy als Fehlerfunktion. Notieren Sie sich den Trainingsfehler anhand der Trainingsdaten und die Vorhersagegenautigkeit mit Hilfe der Testdaten. Plotten Sie den Fehler und die Genauigkeit, um zu überprüfen, ob Ihr Model funktioniert. Es werden Genauigkeiten von ungefähr 92% erwartet.</p>

In [ ]:
input_size = X_train.shape[1]
num_classes = 10

iterations = 50
learning_rate = 1

In [ ]:
# input X: 28x28 grayscale images, the first dimension (None) will index the images in the mini-batch
X_placeholder = tf.placeholder(tf.float32, [None, input_size])
# correct answers will go here
Y_placeholder = tf.placeholder(tf.float32, [None, num_classes])

logits = tf.layers.Dense(10)(X_placeholder)
loss = tf.losses.softmax_cross_entropy(Y_placeholder, logits)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
opt_op = optimizer.minimize(loss)

softmax = tf.nn.softmax(logits)
prediction = tf.argmax(softmax, 1)
labels = tf.argmax(Y_placeholder, 1)
equality = tf.equal(prediction, labels)
accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))

In [ ]:
losses = []
test_losses = []

accuracies = []
test_accuracies = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(iterations):
        # calculating loss value
        l, acc, _ = sess.run([loss, accuracy, opt_op], feed_dict={X_placeholder: X_train, 
                                                                  Y_placeholder: y_train_hot})
        losses.append(l)
        accuracies.append(acc)
        # trainings step
        t_l, t_acc = sess.run([loss, accuracy], feed_dict={X_placeholder: X_test, 
                                                          Y_placeholder: y_test_hot})
        print(l, t_l)
        print(acc, t_acc)
        print("-" * 10)

        test_losses.append(t_l)
        test_accuracies.append(t_acc)

In [ ]:
# TODO Plot the error and accuracy curves
# TODO Plot the training and test error for each trainings iteration
plt.figure(figsize=(10,10))
plt.plot(np.arange(0, iterations), losses, "b", label="train error")
plt.plot(np.arange(0, iterations), test_losses, "r", label="test error")
plt.legend()
plt.show()

In [ ]:
# TODO Plot the training and test error for each trainings iteration
plt.figure(figsize=(10,10))
plt.plot(np.arange(0, iterations), accuracies, "b", label="train acc")
plt.plot(np.arange(0, iterations), test_accuracies, "r", label="test acc")
plt.legend()
plt.show()

<hr />
<h2>Neural Network</h2>

<p>Laut Yann LeCun's Datenbank <a href="http://yann.lecun.com/exdb/mnist/" target="_blank">http://yann.lecun.com/exdb/mnist/</a> sind Neuronale Netzwerke mit 2-3 Schichten besser als einfache Regressionsmodelle. Erweitern Sie Ihr Model zu einem Neuronalen Netzwerk mit mindestens einem Hidden-Layer. Verwenden Sie für diese Schichten die Sigmoid- oder ReLu-Aktivierungsfunktion. Geben Sie wieder den Trainingsfehler und die Testgenauigkeit in einem Diagramm aus. Zu erwarten sind Genauigkeiten von ca. 95%.</p>

In [ ]:
learning_rate = 1
iterations = 100

# input X: 28x28 grayscale images, the first dimension (None) will index the images in the mini-batch
X_placeholder = tf.placeholder(tf.float32, [None, input_size])
# correct answers will go here
Y_placeholder = tf.placeholder(tf.float32, [None, num_classes])

hidden = tf.layers.Dense(200 , activation="relu")(X_placeholder)
logits = tf.layers.Dense(10)(hidden)
loss = tf.losses.softmax_cross_entropy(Y_placeholder, logits)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
opt_op = optimizer.minimize(loss)

softmax = tf.nn.softmax(logits)
prediction = tf.argmax(softmax, 1)
labels = tf.argmax(Y_placeholder, 1)
equality = tf.equal(prediction, labels)
accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))

In [ ]:
losses = []
test_losses = []

accuracies = []
test_accuracies = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(iterations):
        # calculating loss value
        l, acc, _ = sess.run([loss, accuracy, opt_op], feed_dict={X_placeholder: X_train, 
                                                                  Y_placeholder: y_train_hot})
        losses.append(l)
        accuracies.append(acc)
        # trainings step
        t_l, t_acc = sess.run([loss, accuracy], feed_dict={X_placeholder: X_test, 
                                                           Y_placeholder: y_test_hot})

        test_losses.append(t_l)
        test_accuracies.append(t_acc)

In [ ]:
# TODO Plot the error and accuracy curves
# TODO Plot the training and test error for each trainings iteration
plt.figure(figsize=(10,10))
plt.plot(np.arange(0, iterations), losses, "b", label="train error")
plt.plot(np.arange(0, iterations), test_losses, "r", label="test error")
plt.legend()
plt.show()

In [ ]:
# TODO Plot the training and test error for each trainings iteration
plt.figure(figsize=(10,10))
plt.plot(np.arange(0, iterations), accuracies, "b", label="train acc")
plt.plot(np.arange(0, iterations), test_accuracies, "r", label="test acc")
plt.legend()
plt.show()

<hr />

<h2>Abgabe</h2>

Bevor sie das Notebook in Moodle hochladen entfernen sie bitte über "Kernel" -> "Restart and Clear Output" sämtlichen von Python erstellten Inhalt und speichern anschließend das Notebook "File" -> "Save and Checkpoint" erneut ab. Sorgen sie bitte außerdem dafür das im Dateinamen ihr Vor- und Nachname steht, ich empfehle folgende Namensgebung: "05_Tensorflow_Regression_MNIST_VORNAME_NACHNAME.ipynb"